This is my implementation of a new type of Heeger Bergen algorithm for texture synthesis. I double number of filters and use relu to modify algorithm. This still makes invertible pyramid.

# Title

In [ ]:
import numpy as np
import math
import numpy.fft as fft

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
img_orig = mpimg.imread('D4.gif')
img = img_orig[:256, :256] / 255
M = img.shape[0]
N = img.shape[1]
plt.imshow(img, cmap='gray')

# Filters

In [ ]:
def Hj(r, j):
    if(r < 0):
        r = -r #check input
    if(r <= np.pi/(2**(j+1))):
        out = 0
    elif(r <= np.pi/(2**j)):
        out = np.cos((np.pi/2) * np.log2((2**j)*r/np.pi))
    else:
        out = 1
    return out

def Lj(r, j):
    if(r < 0):
        r = -r #check input
    if(r <= np.pi/(2**(j+1))):
        out = 1
    elif(r <= np.pi/(2**j)):
        out = np.cos(np.pi/2 * np.log2((2**(j+1))*r/np.pi))
    else:
        out = 0
    return out

def alpha(Q):
    y = math.factorial(int(2*Q-2))
    return 2**(Q-1) * math.factorial(int(Q-1)) / np.sqrt(Q * y)

def G(q, Q, theta):
    r1 = theta - np.pi * q/Q
    if r1 < -np.pi:
        r1 = r1 + 2*np.pi
    r2 = theta - np.pi * (q-Q)/Q # r2 = theta + np.pi - np.pi * q/Q
    if r2 > np.pi:
        r2 = r2 - 2*np.pi
    x = np.cos(r1)**(Q-1)
    y = np.cos(r2)**(Q-1)
    if (np.abs(r1) <= np.pi/2 and np.abs(r2) <= np.pi/2):
        out = alpha(Q) * (x + y)
    elif (np.abs(r1) >= np.pi/2 and np.abs(r2) <= np.pi/2):
        out = alpha(Q) * (y)
    elif (np.abs(r1) <= np.pi/2 and np.abs(r2) >= np.pi/2):
        out = alpha(Q) * (x)
    else:
        out = 0
    return out

def B(r, theta,j,q, Q):
    return Hj(r,j) * G(q, Q, theta)

def Psi(r, theta, j, q, Q):   
    low = Lj(r, j)
    high = Hj(r, j+1)
    angular = G(q, Q, theta)
    
    return low * high * angular

In [ ]:
def r_no_theta(x,y):
    if(y == 0 and x <= 0):
        r = np.abs(x)
    else:
        r = np.sqrt(x**2 + y**2)
    return r

def r_with_theta(x,y):
    if(y == 0 and x <= 0):
        r = np.abs(x)
        theta = np.pi
    else:
        r = np.sqrt(x**2 + y**2)
        theta = 2*np.arctan(y/(x+r))
    return r, theta

def H_matrix(M,N,j=0): 
    H_mat = np.zeros((M,N), dtype = 'complex')
    r = 0 #input cleanse
    for m in range(-int(M/2), int(M/2)):
        for n in range(-int(N/2), int(N/2)):
            r = r_no_theta(2*np.pi*m/M, 2*np.pi*n/N)
            H_mat[m + int(M/2),n + int(N/2)] = Hj(r,j)
    return H_mat

def L_matrix(M,N,j=0): 
    L_mat = np.zeros((M,N), dtype = 'complex')
    r = 0 #input cleanse
    for m in range(-int(M/2), int(M/2)):
        for n in range(-int(N/2), int(N/2)):
            r = r_no_theta(2*np.pi*m/M,2*np.pi*n/N)
            L_mat[m + int(M/2),n + int(N/2)] = Lj(r,j)
    return L_mat

def B_matrix(M, N, j, q, Q):
    r= 0 #input cleans
    theta=0
    B_mat = np.zeros((M,N), dtype = 'complex')
    for m in range(-int(M/2), int(M/2)):
        for n in range(-int(N/2), int(N/2)):
            r, theta = r_with_theta(2*np.pi*m/M,2*np.pi*n/N)
            B_mat[m,n] = B(r, theta, j, q, Q)
    return B_mat

def psi_matrix(M, N, j, q, Q):
    r = 0 #input cleans
    theta = 0 #input cleans
    psi_mat = np.zeros((M,N), dtype = 'complex')
    for m in range(-int(M/2), int(M/2)):
        for n in range(-int(N/2), int(N/2)):
            r, theta = r_with_theta(2*np.pi*m/M,2*np.pi*n/N)
            psi_mat[m + int(M/2),n + int(N/2)] = Psi(r, theta, j, q, Q)
    return psi_mat


In [ ]:
# Seems that in steerable_pyramid_nodownsample you only use H_all_j[:,:,0], so you can probably eliminate 
# the H_mat_struct function
def H_mat_struct(M,N,J):
    H_all_j = np.zeros((M,N,J), dtype = 'complex')
    for j in range(J):
        H_all_j[:,:,j] = H_matrix(M,N,j)
    return H_all_j

def L_mat_struct(M,N,J):
    L_all_j = np.zeros((M,N,J), dtype = 'complex')
    for j in range(J):  
        L_all_j[:,:,j] = L_matrix(M,N,j)
    return L_all_j

def B_mat_struct(M,N,J,Q):
    B_all_jq = np.zeros((M,N,J,Q), dtype = 'complex')
    for j in range(J):
        for q in range(Q):
            B_all_jq[:,:,j,q] = B_matrix(M, N, j, q, Q)
            
    return B_all_jq

In [ ]:
P = 4
Q = 4

#generate matrix for filters, H_mat doesn't have to have so many elements, might be useful
H_mat = np.zeros((M,N,P), dtype = 'complex')
L_mat = np.zeros((M,N,P+1), dtype = 'complex')
B_mat = np.zeros((M,N,P+1,Q), dtype = 'complex')

#made all the filters
H_mat = H_mat_struct(M,N,P)
L_mat = L_mat_struct(M,N,P+1)
B_mat = B_mat_struct(M,N,P+1,Q)

# Pyramid Algorithm

In [ ]:
def new_pyramid(P, Q, img, H_mat, L_mat, B_mat):
    
    # Get the dimensions the image
    M = img.shape[0]
    N = img.shape[1]
        
    # Initialize pyramid
    pyramid = np.zeros((M,N,P*Q+2,2), dtype = 'complex')

    #find fft of img to work in frequency
    img_fft = np.fft.fft2(img)
    
    # store high frequency residual
    count = 0
    pyramid[:,:,count,0] = np.fft.ifft2(np.fft.fftshift(H_mat[:,:,0]) * img_fft)
    
    # directional wavelet coefficients
    count += 1
    for j in range(P):
        v = np.fft.fftshift(L_mat[:,:,j]) * img_fft # equivalent of step 5 + 6
        for q in range(Q):
            pyramid[:,:,count,0] = np.fft.ifft2(B_mat[:,:,j+1,q] * v) #step 4
            count += 1
    pyramid[:,:,count,0] = np.fft.ifft2(np.fft.fftshift(L_mat[:,:,P]) * img_fft)
    
    #make epsilon = 1 wavelets
    pyramid[:,:,:,1] = -pyramid[:,:,:,0]
    
    #relu
    pyramid[pyramid<0]=0
    
    # return the pyramid
    return pyramid

In [ ]:
# Compute the ReLU wavelet pyramid 
pyra = new_pyramid(P, Q, img, H_mat, L_mat, B_mat)

plt.figure()
plt.figure(figsize = (4*P, 4*Q))
count = 1
for j in range(P):
    for q in range(Q):
        plt.subplot(P, Q, count)
        plt.imshow(np.real(pyra[:,:,count,0]) , extent = [0, M, N, 0], cmap = 'gray')
        plt.colorbar()
        plt.title('j = ' + str(j) + ' q = ' + str(q))
        count = count + 1

plt.figure()
plt.figure(figsize = (4*P, 4*Q))
count = 1
for j in range(P):
    for q in range(Q):
        plt.subplot(P, Q, count)
        plt.imshow(np.real(pyra[:,:,count,1]) , extent = [0, M, N, 0], cmap = 'gray')
        plt.colorbar()
        plt.title('j = ' + str(j) + ' q = ' + str(q))
        count = count + 1

In [ ]:
# Low pass ReLU coefficients

plt.subplot(1,2,1)
plt.imshow(np.real(pyra[:,:,P*Q+1,0]), extent = [0, M, N, 0], cmap = 'gray')
plt.subplot(1,2,2)
plt.imshow(np.real(pyra[:,:,P*Q+1,1]), extent = [0, M, N, 0], cmap = 'gray')
print(np.real(np.sum(pyra[:,:,P*Q+1,1])))

In [ ]:
# High pass ReLU coefficients

plt.subplot(1,2,1)
plt.imshow(np.real(pyra[:,:,0,0]), extent = [0, M, N, 0], cmap = 'gray')
plt.subplot(1,2,2)
plt.imshow(np.real(pyra[:,:,0,1]), extent = [0, M, N, 0], cmap = 'gray')
print(np.real(np.sum(pyra[:,:,1,1])))

In [ ]:
# Check that corresponding coefficient maps, that is epsilon = +1, -1, do not overlap

plt.imshow(np.real(pyra[:,:,1,0] * pyra[:,:,1,1]), extent = [0, M, N, 0], cmap = 'gray')
plt.colorbar();

# Inverse

In [ ]:
def new_pyramid_inv(P, Q, pyramid, H_mat, L_mat, B_mat):
    
    # Initialize the image
    M,N = np.shape(H_mat)[0:2]
    img = np.zeros((M,N))
    
    # Add in the high frequency residual coefficients
    img = img + np.fft.ifft2(np.fft.fft2(pyramid[:,:,0,0]-pyramid[:,:,0,1])* np.fft.fftshift(H_mat[:,:,0]))

    # Add in the low frequency coefficients
    img = img + np.fft.ifft2(np.fft.fft2(pyramid[:,:,P*Q+1,0] - pyramid[:,:,P*Q+1,1]) * np.fft.fftshift(L_mat[:,:,P]))
    
    # Add in the directional wavelet coefficients
    count = 1
    for j in range(P):
        for q in range(Q):
            u = pyramid[:,:,count,0] - pyramid[:,:,count,1]
            u = np.fft.fft2(u) * np.fft.fftshift(L_mat[:,:,j])
            img = img + np.fft.ifft2(u * (B_mat[:,:,j+1,q]))
            count += 1
    
    return img

In [ ]:
inv = new_pyramid_inv(P, Q, pyra, H_mat, L_mat, B_mat)
plt.imshow(np.real(inv), cmap = 'gray')

# Histogram Match

In [ ]:
def hist_match(u, v): #u and v are images of the same size
    v_ravel = v.ravel()
    u_ravel = u.ravel()

    v_sorted = np.sort(v_ravel) # sort in ascending order
    # u_sorted = np.sort(u_ravel)

    tau = np.argsort(v_ravel)
    sigma = np.argsort(u_ravel)

    u_new_vec = np.zeros(v_ravel.shape[0]) #new vector after matching
    for i in range(v_ravel.shape[0]):
#         u_new_vec[tau[i]] = v_ravel[sigma[i]] #replace like in paper
        u_new_vec[sigma[i]] = v_ravel[tau[i]] #replace like in paper

    u_new = np.reshape(u_new_vec, (u.shape[0], u.shape[1]), order = 'C') 
    return u_new

# White Noise

In [ ]:
def gen_white_noise(img):
    noise = np.zeros((img.shape[0], img.shape[1]), dtype = 'float')
    for n in range(img.shape[0]):
        for m in range(img.shape[1]):
            noise[n][m] = np.random.normal(0, 0.5)
    return noise

# Heeger Bergen + Test Case

In [ ]:
def heeger_bergen(img, P, Q, iterations, H_mat, L_mat, B_mat):
    
    # Get the dimensions of the image
    M,N = np.shape(img)
    
    # Generate the filters
#     H_mat = H_mat_struct(M,N,P)
#     L_mat = L_mat_struct(M,N,P+1)
#     B_mat = B_mat_struct(M,N,P+1,Q)
    
    # Store each noise iteration
    noise = np.zeros((M,N,iterations+1))
    
    # Synthesis algorithm
    img_pyra = new_pyramid(P, Q, img, H_mat, L_mat, B_mat)
    img_pyra = np.real(img_pyra)
    noise[:,:,0] = gen_white_noise(img)
    noise[:,:,0] = hist_match(noise[:,:,0], img)
    for n in range(iterations):
        noise_pyra = new_pyramid(P, Q, noise[:,:,n], H_mat, L_mat, B_mat)
        for k in range(2):
            for j in range(P*Q+2):
                noise_pyra[:,:,j,k] = hist_match(np.real(noise_pyra[:,:,j,k]), img_pyra[:,:,j,k])
            noise[:,:,n+1] = np.real(new_pyramid_inv(P, Q, noise_pyra, H_mat, L_mat, B_mat))
            noise[:,:,n+1] = hist_match(noise[:,:,n+1], img)  
    return noise

In [ ]:
num_iter = 5
texture = heeger_bergen(img, P, Q, num_iter, H_mat, L_mat, B_mat)
plt.imshow(np.real(texture[:,:,num_iter]), cmap = 'gray')
#plt.imshow(np.real(texture - img))
plt.colorbar()

In [ ]:
# Display and compare

for n in range(num_iter + 1):
    
    plt.figure(figsize = (18,5))
    
    plt.subplot(1,3,1)
    plt.imshow(img, cmap = 'gray')
    plt.colorbar()
    plt.title('Original image')

    plt.subplot(1,3,2)
    plt.imshow(texture[:,:,n], cmap = 'gray')
    plt.colorbar()
    plt.title('Synthesized texture')

    plt.subplot(1,3,3)
    plt.imshow(np.abs(img - texture[:,:,n]), cmap = 'gray')
    plt.colorbar()
    plt.title('Difference of images')